# IES

# Investigadores

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.ieschile.cl/investigadores"

try:
    response = requests.get(url)
    response.raise_for_status()  # Lanza una excepción para códigos de estado HTTP no exitosos (4xx o 5xx)

    soup = BeautifulSoup(response.content, 'html.parser')

    data = []
    for item in soup.find_all('div', class_='ourResearchers_itemContainer__vy2TR'):
        name = item.find('div', class_='ourResearchers_nameItem__q7OHQ').text.strip()
        link = item.find_parent('a')['href']
        full_link = "https://www.ieschile.cl/" + link # <--- Agregar el dominio base
        data.append({'Nombre': name, 'Enlace': full_link})

    investigadores = pd.DataFrame(data)

except requests.exceptions.RequestException as e:
    print(f"Error al acceder a la URL: {e}")
except AttributeError as e:
    print(f"Error al analizar el HTML: {e}.  Asegúrese de que los selectores CSS sean correctos.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

investigadores

,Nombre,Enlace
0,Claudio Alvarado R.,https://www.ieschile.cl/investigadores/claudio...
1,Joaquín Castillo V.,https://www.ieschile.cl/investigadores/joaquin...
2,Josefina Araos B.,https://www.ieschile.cl/investigadores/josefin...
3,Pablo Ortúzar M.,https://www.ieschile.cl/investigadores/pablo-o...
4,Álvaro Vergara N.,https://www.ieschile.cl/investigadores/alvaro-...
5,Catalina Siles V.,https://www.ieschile.cl/investigadores/catalin...
6,Daniel Mansuy H.,https://www.ieschile.cl/investigadores/daniel-...
7,Manfred Svensson,https://www.ieschile.cl/investigadores/manfred...
8,Guillermo Pérez C.,https://www.ieschile.cl/investigadores/guiller...


In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
    service = Service()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

driver = create_driver(headless=False)

data = []
BASE_URL = "https://www.ejemplo.com"  # Cambia esto según la URL base real

for _, row in investigadores[:1].iterrows():
    enlace = row['Enlace']
    nombre = row['Nombre']

    driver.get(enlace)
    time.sleep(3)  # Espera para que la página cargue completamente

    tabs = driver.find_elements(By.XPATH, "//div[@class='ant-tabs-nav-list']/div/div")
    
    for tab in tabs:
        tab.click()
        time.sleep(3)  # Esperar para asegurar que la pestaña se cargue

        tarjetas = driver.find_elements(By.XPATH, "//div[contains(@class, 'investigadores_containerEsp__C3QGu')]")
        print(f"Categoría: {tab.text} - Elementos encontrados: {len(tarjetas)}")

        for tarjeta in tarjetas:
            try:
                categoria = tab.text  

                # Extraer título
                title_element = tarjeta.find_element(By.XPATH, ".//div[contains(@style, 'font-family: RalewayBold')] | .//a//div[contains(@style, 'font-family: RalewayBold')]")
                titulo = title_element.text.strip()

                # Extraer fecha
                try:
                    fecha_element = tarjeta.find_element(By.XPATH, ".//div[contains(@style, 'font-family: InterLight')]")
                    fecha = fecha_element.text.strip()
                except:
                    fecha = "No disponible"

                # Extraer enlace usando JavaScript para evitar problemas de visibilidad
                try:
                    link_element = tarjeta.find_element(By.XPATH, ".//a")
                    link = driver.execute_script("return arguments[0].href;", link_element)  # Obtener `href` directamente

                    if not link.startswith("http"):  # Corregir enlaces relativos
                        link = BASE_URL + link
                except:
                    link = "No disponible"

                data.append({
                    'Investigador': nombre,
                    'Categoría': categoria,
                    'Título': titulo,
                    'Fecha': fecha,
                    'Enlace': link
                })
            except Exception as e:
                print(f"Error al extraer datos de una tarjeta: {e}")
                print(f"Tarjeta HTML: {tarjeta.get_attribute('innerHTML')}")  # Para debugging

df_inv = pd.DataFrame(data)
driver.quit()
df_inv

Categoría: OPINIÓN - Elementos encontrados: 31
Categoría: VIDEOS - Elementos encontrados: 35
Categoría: LIBROS - Elementos encontrados: 42
Categoría: CLAVES E INFORMES - Elementos encontrados: 46


,Investigador,Categoría,Título,Fecha,Enlace
0,Claudio Alvarado R.,OPINIÓN,"Pensiones, empleo y ciudadanía",20 de Enero del 2025,No disponible
1,Claudio Alvarado R.,OPINIÓN,Elecciones 2025: La oportunidad histórica de l...,10 de Enero del 2025,No disponible
2,Claudio Alvarado R.,OPINIÓN,La agonía del octubrismo,30 de Diciembre del 2024,No disponible
3,Claudio Alvarado R.,OPINIÓN,Ni Allende ni Aylwin,30 de Diciembre del 2024,No disponible
4,Claudio Alvarado R.,OPINIÓN,"""Habitar el cargo""",27 de Diciembre del 2024,No disponible
...,...,...,...,...,...
149,Claudio Alvarado R.,CLAVES E INFORMES,,No disponible,No disponible
150,Claudio Alvarado R.,CLAVES E INFORMES,El proceso constitucional 2.0 versus la fallid...,No disponible,No disponible
151,Claudio Alvarado R.,CLAVES E INFORMES,Objeción de conciencia institucional: 4 claves...,No disponible,No disponible
152,Claudio Alvarado R.,CLAVES E INFORMES,Limitaciones y amenazas a la libertad de educa...,No disponible,No disponible


In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
    service = Service()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

driver = create_driver(headless=False)
url = "https://www.ieschile.cl/investigadores/claudio-alvarado"
driver.get(url)
time.sleep(5)  # Esperar a que cargue la página

# Lista para almacenar los datos
data = []

try:
    # Encontrar las pestañas
    categorias = driver.find_elements(By.CSS_SELECTOR, "div.ant-tabs-tab")

    if not categorias:
        print("⚠️ No se encontraron pestañas. Verifica el selector.")
    else:
        print(f"✅ {len(categorias)} pestañas encontradas.")

    # Iterar sobre las pestañas y hacer clic en cada una para cargar su contenido
    for i, categoria_tab in enumerate(categorias):
        try:
            # Obtener el nombre de la categoría
            categoria = categoria_tab.text.strip()
            print(f"📌 Abriendo categoría: {categoria}")

            # Hacer scroll hasta la pestaña y hacer clic
            driver.execute_script("arguments[0].scrollIntoView();", categoria_tab)
            driver.execute_script("arguments[0].click();", categoria_tab)
            time.sleep(2)  # Esperar a que el contenido se cargue

            # Esperar hasta que el panel se actualice con nuevo contenido
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.ant-tabs-tabpane-active"))
            )

            # Buscar el panel activo
            panel = driver.find_element(By.CSS_SELECTOR, "div.ant-tabs-tabpane-active")
            tarjetas = panel.find_elements(By.CSS_SELECTOR, "div > div")

            for tarjeta in tarjetas:
                try:
                    # Intentar extraer los datos de cada tarjeta
                    enlace = tarjeta.find_element(By.TAG_NAME, "a").get_attribute("href") if tarjeta.find_elements(By.TAG_NAME, "a") else None
                    titulo = tarjeta.find_element(By.CSS_SELECTOR, "div[style*='font-family: RalewayBold']").text if tarjeta.find_elements(By.CSS_SELECTOR, "div[style*='font-family: RalewayBold']") else None
                    fecha = tarjeta.find_element(By.CSS_SELECTOR, "div[style*='font-family: InterLight']").text if tarjeta.find_elements(By.CSS_SELECTOR, "div[style*='font-family: InterLight']") else None
                    imagen = tarjeta.find_element(By.TAG_NAME, "img").get_attribute("src") if tarjeta.find_elements(By.TAG_NAME, "img") else None

                    # Solo agregar la tarjeta si tiene los datos
                    if enlace and titulo and fecha and imagen:
                        data.append([categoria, titulo, fecha, enlace, imagen])

                except Exception as e:
                    print(f"⚠️ Error extrayendo datos de tarjeta en '{categoria}': {e}")

        except Exception as e:
            print(f"⚠️ No se pudo abrir la pestaña '{categoria}': {e}")

except Exception as e:
    print(f"⚠️ Error: No se encontraron categorías - {e}")

driver.quit()

# Crear DataFrame con los datos obtenidos
df_inv = pd.DataFrame(data, columns=["Categoría", "Título", "Fecha", "Enlace", "Imagen"])

# Mostrar las primeras filas del DataFrame
df_inv

✅ 4 pestañas encontradas.
📌 Abriendo categoría: OPINIÓN
📌 Abriendo categoría: VIDEOS
📌 Abriendo categoría: LIBROS
📌 Abriendo categoría: CLAVES E INFORMES


,Categoría,Título,Fecha,Enlace,Imagen
0,OPINIÓN,"Pensiones, empleo y ciudadanía",20 de Enero del 2025,https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...
1,OPINIÓN,"Pensiones, empleo y ciudadanía",20 de Enero del 2025,https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...
2,OPINIÓN,Elecciones 2025: La oportunidad histórica de l...,10 de Enero del 2025,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...
3,OPINIÓN,La agonía del octubrismo,30 de Diciembre del 2024,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...
4,OPINIÓN,Ni Allende ni Aylwin,30 de Diciembre del 2024,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...
5,OPINIÓN,"""Habitar el cargo""",27 de Diciembre del 2024,https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...
6,OPINIÓN,Juan Antonio Widow (1935-2024),23 de Diciembre del 2024,https://www.ieschile.cl/columna/juan-antonio-w...,https://www.ieschile.cl/_next/image?url=https%...
7,OPINIÓN,Falso dilema,02 de Diciembre del 2024,https://www.ieschile.cl/columna/falso-dilema,https://www.ieschile.cl/_next/image?url=https%...
8,OPINIÓN,La incomodidad de Claudio Orrego,18 de Noviembre del 2024,https://www.ieschile.cl/columna/la-incomodidad...,https://www.ieschile.cl/_next/image?url=https%...
9,OPINIÓN,Hijos sin padre,11 de Noviembre del 2024,https://www.ieschile.cl/columna/hijos-sin-padre,https://www.ieschile.cl/_next/image?url=https%...


Videos

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Crea una instancia del driver de Selenium (asegúrate de tener configurado el driver adecuado)
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Si no quieres abrir la ventana del navegador
    return webdriver.Chrome(options=options)

driver = create_driver()

# Accede a la página web
driver.get("https://www.ieschile.cl/investigadores/claudio-alvarado")  # URL de la página que deseas acceder

# Espera a que el enlace "Videos" esté presente y haz clic en él
wait = WebDriverWait(driver, 10)  # Tiempo de espera incrementado a 10 segundos
videos_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#rc-tabs-0-tab-2")))  # Ajusta el selector CSS si es necesario
videos_button.click()

# Espera a que el contenido de videos cargue
time.sleep(2)  # Ajusta el tiempo según la velocidad de carga de la página

# Extrae el HTML de la página
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encuentra todas las tarjetas de video
video_cards = soup.select("#rc-tabs-0-panel-2 > div .ant-col")

# Extrae los enlaces y títulos de cada video
video_links = []
for card in video_cards:
    link = card.find('a', href=True)
    if link:
        video_url = link['href']
        title = card.find('div', style=True)
        if title:
            video_links.append({'title': title.get_text(strip=True), 'url': video_url})

# Crea un DataFrame a partir de los resultados
df = pd.DataFrame(video_links)

# Cierra el navegador
driver.quit()

df

,title,url
0,"""Niñez y transexualidad. Reflexiones críticas ...",https://www.youtube.com/watch?v=IXPn5PwoD58
1,Entrevista a Claudio Alvarado en Emol TV (07-0...,https://www.youtube.com/watch?v=XuaHMWd9DHk
2,Entrevista a Claudio Alvarado en EmolTV (03-10...,https://www.youtube.com/watch?v=XpkwoALLLn0
3,Entrevista a Claudio Alvarado sobre la nueva i...,https://www.youtube.com/watch?v=AcGR34NGswM


Libros

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Crea una instancia del driver de Selenium
def create_driver():
    options = webdriver.ChromeOptions()
    #options.add_argument('--headless')  # Si no quieres abrir la ventana del navegador
    return webdriver.Chrome(options=options)

driver = create_driver()

# Accede a la página web
driver.get("https://www.ieschile.cl/investigadores/claudio-alvarado")  # URL de la página que deseas acceder

# Espera a que el enlace "Libros" esté presente y haz clic en él
wait = WebDriverWait(driver, 15)  # Aumentar el tiempo de espera
try:
    books_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#rc-tabs-0-tab-3")))  # Selector para la pestaña de libros
    books_button.click()
except Exception as e:
    print(f"Error al intentar hacer clic en la pestaña de libros: {e}")

# Espera a que el contenido de libros cargue
time.sleep(3)  # Aumentar el tiempo de espera para asegurarse de que se cargue el contenido

# Extrae el HTML de la página
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encuentra todas las tarjetas de libros usando el selector CSS con nth-child
book_links = []
for i in range(1, 11):  # Puedes ajustar el rango según la cantidad de tarjetas en la página
    card_selector = f"#rc-tabs-0-panel-3 > div > div:nth-child({i})"
    card = soup.select_one(card_selector)
    if card:
        link = card.find('a', href=True)
        if link:
            book_url = link['href']
            title = card.find('div', style=True)
            if title:
                book_links.append({'title': title.get_text(strip=True), 'url': book_url})

# Crea un DataFrame a partir de los resultados
df_books = pd.DataFrame(book_links)

# Cierra el navegador
driver.quit()

# Muestra el DataFrame
df_books

,title,url
0,Primera persona singular. Reflexiones entorna ...,https://tienda.ieschile.cl/products/primera_pe...
1,Curso de filosofía política,https://tienda.ieschile.cl/products/curso_de_f...
2,1943: La crisis del humanismo cristiano,https://tienda.ieschile.cl/products/1943-la-cr...
3,El pueblo olvidado. Una crítica a la comprensi...,https://tienda.ieschile.cl/products/el-pueblo-...
4,La ilusión constitucional,https://tienda.ieschile.cl/products/la_ilusion...
5,Imaginar la República: Reflexiones sobre El fe...,https://tienda.ieschile.cl/products/imaginar_l...
6,Tensión Constituyente,https://tienda.ieschile.cl/collections/editori...


Claves e informes

In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Crea una instancia del driver de Selenium
def create_driver():
    options = webdriver.ChromeOptions()
    #options.add_argument('--headless')  # Si no quieres abrir la ventana del navegador
    return webdriver.Chrome(options=options)

driver = create_driver()

# Accede a la página web
driver.get("https://www.ieschile.cl/investigadores/claudio-alvarado")  # URL de la página que deseas acceder

# Espera a que el enlace "Claves e informes" esté presente y haz clic en él
wait = WebDriverWait(driver, 15)  # Aumentar el tiempo de espera
try:
    keys_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#rc-tabs-0-tab-5")))  # Selector para la pestaña de claves e informes
    keys_button.click()
except Exception as e:
    print(f"Error al intentar hacer clic en la pestaña de claves e informes: {e}")

# Espera a que el contenido de claves e informes cargue
time.sleep(3)  # Aumentar el tiempo de espera para asegurarse de que se cargue el contenido

# Extrae el HTML de la página
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encuentra todas las tarjetas de claves e informes usando el selector CSS con nth-child
key_reports_links = []
for i in range(1, 11):  # Puedes ajustar el rango según la cantidad de tarjetas en la página
    card_selector = f"#rc-tabs-0-panel-5 > div > div:nth-child({i})"
    card = soup.select_one(card_selector)
    if card:
        link = card.find('a', href=True)
        if link:
            key_report_url = link['href']
            title = card.find('div', style=True)
            if title:
                key_reports_links.append({'title': title.get_text(strip=True), 'url': key_report_url})

# Crea un DataFrame a partir de los resultados
df_keys_reports = pd.DataFrame(key_reports_links)

# Cierra el navegador
driver.quit()

# Muestra el DataFrame
df_keys_reports

,title,url
0,El proceso constitucional 2.0 versus la fallid...,https://multi-media.nyc3.digitaloceanspaces.co...
1,Objeción de conciencia institucional: 4 claves...,https://multi-media.nyc3.digitaloceanspaces.co...
2,Limitaciones y amenazas a la libertad de educa...,https://multi-media.nyc3.digitaloceanspaces.co...
3,Nueva constitución y derechos sociales: 5 clav...,https://multi-media.nyc3.digitaloceanspaces.co...


Todo junto

In [56]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup

# Crea una instancia del driver de Selenium
def create_driver(headless=False):
    options = Options()
    if headless:
        options.add_argument("--headless")
        options.add_argument("--disable-gpu")
    service = Service()
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Inicia el driver
driver = create_driver(headless=False)
url = "https://www.ieschile.cl/investigadores/claudio-alvarado"
driver.get(url)
time.sleep(5)  # Espera a que cargue la página

# Lista para almacenar los datos
data = []

# Función para extraer datos de la pestaña "Opinión"
def extract_opinion_data():
    try:
        # Extrae el HTML de la página de Opinión
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Extrae las tarjetas de opinión
        panel = driver.find_element(By.CSS_SELECTOR, "div.ant-tabs-tabpane-active")
        tarjetas = panel.find_elements(By.CSS_SELECTOR, "div > div")

        for tarjeta in tarjetas:
            try:
                enlace = tarjeta.find_element(By.TAG_NAME, "a").get_attribute("href") if tarjeta.find_elements(By.TAG_NAME, "a") else None
                titulo = tarjeta.find_element(By.CSS_SELECTOR, "div[style*='font-family: RalewayBold']").text if tarjeta.find_elements(By.CSS_SELECTOR, "div[style*='font-family: RalewayBold']") else None
                fecha = tarjeta.find_element(By.CSS_SELECTOR, "div[style*='font-family: InterLight']").text if tarjeta.find_elements(By.CSS_SELECTOR, "div[style*='font-family: InterLight']") else None
                imagen = tarjeta.find_element(By.TAG_NAME, "img").get_attribute("src") if tarjeta.find_elements(By.TAG_NAME, "img") else None

                # Solo agregar si tiene todos los datos necesarios
                if enlace and titulo and fecha and imagen:
                    data.append(["Opinión", titulo, fecha, enlace, imagen])
            except Exception as e:
                print(f"⚠️ Error extrayendo datos de tarjeta en 'Opinión': {e}")
    except Exception as e:
        print(f"⚠️ No se pudo extraer los datos de 'Opinión': {e}")

# Función para extraer datos de la pestaña "Videos"
def extract_videos_data():
    try:
        # Hacer clic en la pestaña "Videos"
        wait = WebDriverWait(driver, 10)
        videos_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#rc-tabs-0-tab-2")))
        videos_button.click()
        
        # Espera que el contenido de videos cargue
        time.sleep(2)

        # Extrae el HTML de la página
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Encuentra las tarjetas de video
        video_cards = soup.select("#rc-tabs-0-panel-2 > div .ant-col")

        # Extrae los enlaces y títulos de cada video
        for card in video_cards:
            link = card.find('a', href=True)
            if link:
                video_url = link['href']
                title = card.find('div', style=True)
                if title:
                    data.append(["Videos", title.get_text(strip=True), video_url])
    except Exception as e:
        print(f"⚠️ Error extrayendo datos de 'Videos': {e}")

# Función para extraer datos de la pestaña "Libros"
def extract_books_data():
    try:
        # Hacer clic en la pestaña "Libros"
        wait = WebDriverWait(driver, 15)  # Aumentamos el tiempo de espera
        books_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#rc-tabs-0-tab-3")))  # Selector para la pestaña de libros
        books_button.click()
        
        # Espera a que el contenido de libros cargue
        time.sleep(3)  # Aumentamos el tiempo de espera para asegurarnos de que se cargue el contenido

        # Extrae el HTML de la página
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Encuentra todas las tarjetas de libros usando el selector CSS con nth-child
        book_links = []
        for i in range(1, 11):  # Puedes ajustar el rango según la cantidad de tarjetas en la página
            card_selector = f"#rc-tabs-0-panel-3 > div > div:nth-child({i})"
            card = soup.select_one(card_selector)
            if card:
                link = card.find('a', href=True)
                if link:
                    book_url = link['href']
                    title = card.find('div', style=True)
                    if title:
                        book_links.append({'Título': title.get_text(strip=True), 'Enlace': book_url})

        # Agregar los libros extraídos a los datos
        for book in book_links:
            data.append(["Libros", book['Título'], book['Enlace']])
    except Exception as e:
        print(f"⚠️ Error extrayendo datos de 'Libros': {e}")

# Función para extraer datos de la pestaña "Claves e informes"
def extract_keys_reports_data():
    try:
        # Hacer clic en la pestaña "Claves e informes"
        wait = WebDriverWait(driver, 15)  # Aumentamos el tiempo de espera
        keys_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#rc-tabs-0-tab-5")))  # Selector para la pestaña de claves e informes
        keys_button.click()

        # Espera a que el contenido de claves e informes cargue
        time.sleep(3)  # Aumentar el tiempo de espera para asegurarse de que se cargue el contenido

        # Extrae el HTML de la página
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Encuentra todas las tarjetas de claves e informes usando el selector CSS con nth-child
        key_reports_links = []
        for i in range(1, 11):  # Puedes ajustar el rango según la cantidad de tarjetas en la página
            card_selector = f"#rc-tabs-0-panel-5 > div > div:nth-child({i})"
            card = soup.select_one(card_selector)
            if card:
                link = card.find('a', href=True)
                if link:
                    key_report_url = link['href']
                    title = card.find('div', style=True)
                    if title:
                        key_reports_links.append({'Título': title.get_text(strip=True), 'Enlace': key_report_url})

        # Agregar los informes y claves extraídos a los datos
        for key_report in key_reports_links:
            data.append(["Claves e informes", key_report['Título'], key_report['Enlace']])
    except Exception as e:
        print(f"⚠️ Error extrayendo datos de 'Claves e informes': {e}")

# Extraer datos de "Opinión"
extract_opinion_data()

# Extraer datos de "Videos"
extract_videos_data()

# Extraer datos de "Libros"
extract_books_data()

# Extraer datos de "Claves e informes"
extract_keys_reports_data()

# Cerrar el driver
driver.quit()

# Crear DataFrame con los datos obtenidos
df_inv = pd.DataFrame(data, columns=["Categoría", "Título", "Fecha/URL", "Enlace", "Imagen"])

# Mostrar las primeras filas del DataFrame
df_inv

,Categoría,Título,Fecha/URL,Enlace,Imagen
0,Opinión,"Pensiones, empleo y ciudadanía",20 de Enero del 2025,https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...
1,Opinión,"Pensiones, empleo y ciudadanía",20 de Enero del 2025,https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...
2,Opinión,Elecciones 2025: La oportunidad histórica de l...,10 de Enero del 2025,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...
3,Opinión,La agonía del octubrismo,30 de Diciembre del 2024,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...
4,Opinión,Ni Allende ni Aylwin,30 de Diciembre del 2024,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...
5,Opinión,"""Habitar el cargo""",27 de Diciembre del 2024,https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...
6,Opinión,Juan Antonio Widow (1935-2024),23 de Diciembre del 2024,https://www.ieschile.cl/columna/juan-antonio-w...,https://www.ieschile.cl/_next/image?url=https%...
7,Opinión,Falso dilema,02 de Diciembre del 2024,https://www.ieschile.cl/columna/falso-dilema,https://www.ieschile.cl/_next/image?url=https%...
8,Opinión,La incomodidad de Claudio Orrego,18 de Noviembre del 2024,https://www.ieschile.cl/columna/la-incomodidad...,https://www.ieschile.cl/_next/image?url=https%...
9,Opinión,Hijos sin padre,11 de Noviembre del 2024,https://www.ieschile.cl/columna/hijos-sin-padre,https://www.ieschile.cl/_next/image?url=https%...


In [57]:
# Función para extraer datos de las diferentes pestañas para cada autor
def extract_author_data(investigador):
    try:
        # Recoger el nombre del autor
        autor = investigador['Nombre']

        # Iniciar el driver
        driver.get(investigador['Enlace'])
        time.sleep(5)  # Esperar a que cargue la página

        # Lista temporal para almacenar los datos de este autor
        temp_data = []

        # Extraer datos de "Opinión"
        extract_opinion_data()

        # Extraer datos de "Videos"
        extract_videos_data()

        # Extraer datos de "Libros"
        extract_books_data()

        # Extraer datos de "Claves e informes"
        extract_keys_reports_data()

        # Modificar las categorías con el nombre del autor
        for row in data:
            row.insert(1, autor)  # Insertar el nombre del autor en la posición correcta (después de la categoría)

        # Agregar los resultados al DataFrame global
        for row in data:
            temp_data.append(row)

        # Limpiar los datos para el siguiente autor
        data.clear()

        return temp_data

    except Exception as e:
        print(f"⚠️ Error extrayendo datos del autor {autor}: {e}")
        return []

# Lista para almacenar todos los datos extraídos
all_data = []

driver = create_driver(headless=False)

# Iterar sobre el DataFrame de investigadores y extraer datos para cada uno
for index, investigador in investigadores.iterrows():
    autor_data = extract_author_data(investigador)
    all_data.extend(autor_data)

driver.quit()

# Crear DataFrame con los datos obtenidos
df_final_investigadores = pd.DataFrame(all_data, columns=["Categoría", "Autor", "Título", "Fecha/URL", "Enlace", "Imagen"])

# Mostrar las primeras filas del DataFrame final
df_final_investigadores

,Categoría,Autor,Título,Fecha/URL,Enlace,Imagen
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",20 de Enero del 2025,https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...
1,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",20 de Enero del 2025,https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...
2,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,10 de Enero del 2025,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...
3,Opinión,Claudio Alvarado R.,La agonía del octubrismo,30 de Diciembre del 2024,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...
4,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,30 de Diciembre del 2024,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...
...,...,...,...,...,...,...
324,Claves e informes,Guillermo Pérez C.,¿Un Estado de bienestar para Chile?: 5 claves ...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
325,Claves e informes,Guillermo Pérez C.,Mecanismos de democracia directa y nueva const...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
326,Claves e informes,Guillermo Pérez C.,Descentralización y gobernadores regionales: 5...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
327,Claves e informes,Guillermo Pérez C.,Migración en Chile: 5 claves para el debate,https://multi-media.nyc3.digitaloceanspaces.co...,None,None


In [60]:
import pandas as pd

# ... (tu DataFrame df_final_investigadores)

# Función para separar fecha y enlace, conservando enlaces existentes (CORREGIDA)
def separar_fecha_url(row):
    valor = row['Fecha/URL']
    enlace_existente = row['Enlace']

    if isinstance(valor, str) and valor.startswith('http'):
        return None, valor  # Si comienza con http, es un enlace NUEVO
    return valor, enlace_existente  # Si no, es una fecha, se conserva el enlace existente


df_final_investigadores2 = df_final_investigadores.copy()

# Aplicar la función para crear las columnas "Fecha" y "Enlace"
df_final_investigadores2[['Fecha', 'Enlace']] = df_final_investigadores2.apply(separar_fecha_url, axis=1).tolist()

# Eliminar la columna original "Fecha/URL"
df_final_investigadores2.drop('Fecha/URL', axis=1, inplace=True)

# Imprimir el DataFrame resultante
df_final_investigadores2

,Categoría,Autor,Título,Enlace,Imagen,Fecha
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,20 de Enero del 2025
1,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,20 de Enero del 2025
2,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...,10 de Enero del 2025
3,Opinión,Claudio Alvarado R.,La agonía del octubrismo,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024
4,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024
...,...,...,...,...,...,...
324,Claves e informes,Guillermo Pérez C.,¿Un Estado de bienestar para Chile?: 5 claves ...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
325,Claves e informes,Guillermo Pérez C.,Mecanismos de democracia directa y nueva const...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
326,Claves e informes,Guillermo Pérez C.,Descentralización y gobernadores regionales: 5...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
327,Claves e informes,Guillermo Pérez C.,Migración en Chile: 5 claves para el debate,https://multi-media.nyc3.digitaloceanspaces.co...,None,None


In [61]:
df_final_investigadores3 = df_final_investigadores2.drop_duplicates()
df_final_investigadores3

,Categoría,Autor,Título,Enlace,Imagen,Fecha
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,20 de Enero del 2025
2,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...,10 de Enero del 2025
3,Opinión,Claudio Alvarado R.,La agonía del octubrismo,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024
4,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024
5,Opinión,Claudio Alvarado R.,"""Habitar el cargo""",https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...,27 de Diciembre del 2024
...,...,...,...,...,...,...
324,Claves e informes,Guillermo Pérez C.,¿Un Estado de bienestar para Chile?: 5 claves ...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
325,Claves e informes,Guillermo Pérez C.,Mecanismos de democracia directa y nueva const...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
326,Claves e informes,Guillermo Pérez C.,Descentralización y gobernadores regionales: 5...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
327,Claves e informes,Guillermo Pérez C.,Migración en Chile: 5 claves para el debate,https://multi-media.nyc3.digitaloceanspaces.co...,None,None


In [62]:
df_final_investigadores3.to_parquet("IES-corpus.parquet",index=False)

In [2]:
import pandas as pd

df_final_investigadores_cargado = pd.read_parquet(r"D:\Escritorio D\Scrap Rodrigo\Parte 3\IES-corpus.parquet")
df_final_investigadores_cargado

,Categoría,Autor,Título,Enlace,Imagen,Fecha
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,20 de Enero del 2025
1,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...,10 de Enero del 2025
2,Opinión,Claudio Alvarado R.,La agonía del octubrismo,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024
3,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024
4,Opinión,Claudio Alvarado R.,"""Habitar el cargo""",https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...,27 de Diciembre del 2024
...,...,...,...,...,...,...
270,Claves e informes,Guillermo Pérez C.,¿Un Estado de bienestar para Chile?: 5 claves ...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
271,Claves e informes,Guillermo Pérez C.,Mecanismos de democracia directa y nueva const...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
272,Claves e informes,Guillermo Pérez C.,Descentralización y gobernadores regionales: 5...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None
273,Claves e informes,Guillermo Pérez C.,Migración en Chile: 5 claves para el debate,https://multi-media.nyc3.digitaloceanspaces.co...,None,None


# Corpus

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Suponiendo que ya tienes tu DataFrame 'df_final_investigadores_cargado'
# df_final_investigadores_cargado = pd.read_csv("ruta_a_tu_archivo.csv")

def obtener_corpus(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Asegura que la solicitud fue exitosa
        soup = BeautifulSoup(response.text, 'html.parser')

        # Primero intentamos obtener el corpus desde el primer selector
        corpus = []
        div_contenido = soup.select_one('body > div > main > div.ant-row.ant-row-center.css-42nv3w > div.ant-col.ant-col-xs-22.ant-col-md-16.css-42nv3w > div')
        
        if div_contenido:
            # Extraemos los párrafos dentro del div
            parrafos = div_contenido.find_all('p')
            for p in parrafos:
                corpus.append(p.get_text(strip=True))  # Agregar solo el texto, sin etiquetas
        return " ".join(corpus)
    
    except requests.exceptions.RequestException as e:
        print(f"Error al procesar la URL {url}: {e}")
        return None

df_final_investigadores_cargado2 = df_final_investigadores_cargado[:5].copy()

df_final_investigadores_cargado2 = df_final_investigadores_cargado.sample(n=10)

df_final_investigadores_cargado2 = df_final_investigadores_cargado.copy()

# Aplica la función de extracción a cada enlace en el DataFrame
df_final_investigadores_cargado2['corpus'] = df_final_investigadores_cargado2['Enlace'].apply(obtener_corpus)

# Ahora puedes revisar el DataFrame con la nueva columna 'corpus'
df_final_investigadores_cargado2

,Categoría,Autor,Título,Enlace,Imagen,Fecha,corpus
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,20 de Enero del 2025,"68%. Ese es el porcentaje que, según la encues..."
1,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...,10 de Enero del 2025,"Desde el punto de vista político, el nuevo año..."
2,Opinión,Claudio Alvarado R.,La agonía del octubrismo,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024,Las izquierdas jamás anticiparon cuán mal enve...
3,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024,Si la idea de una superior “escala de valores ...
4,Opinión,Claudio Alvarado R.,"""Habitar el cargo""",https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...,27 de Diciembre del 2024,"Señor Director: En su última cuenta pública, a..."
...,...,...,...,...,...,...,...
270,Claves e informes,Guillermo Pérez C.,¿Un Estado de bienestar para Chile?: 5 claves ...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,
271,Claves e informes,Guillermo Pérez C.,Mecanismos de democracia directa y nueva const...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,
272,Claves e informes,Guillermo Pérez C.,Descentralización y gobernadores regionales: 5...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,
273,Claves e informes,Guillermo Pérez C.,Migración en Chile: 5 claves para el debate,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,


In [7]:
def obtener_corpus(url, existing_corpus):
    if existing_corpus and existing_corpus != "":
        return existing_corpus  # Retorna el valor actual de corpus si no está vacío
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Asegura que la solicitud fue exitosa
        soup = BeautifulSoup(response.text, 'html.parser')

        corpus = []

        # Primero intentamos obtener el corpus desde el primer selector
        div_contenido = soup.select_one('body > div > main > div.ant-row.ant-row-center.css-42nv3w > div.ant-col.ant-col-xs-22.ant-col-md-16.css-42nv3w > div')
        
        if div_contenido:
            # Extraemos los párrafos dentro del div
            parrafos = div_contenido.find_all('p')
            for p in parrafos:
                corpus.append(p.get_text(strip=True))  # Agregar solo el texto, sin etiquetas

        # Añadir extracción adicional usando el nuevo selector
        parrafos_adicionales = soup.select('p:nth-child(1), p:nth-child(2), p:nth-child(3)')  # Aquí puedes ajustarlo para otros párrafos específicos
        for p in parrafos_adicionales:
            corpus.append(p.get_text(strip=True))

        return " ".join(corpus)
    
    except requests.exceptions.RequestException as e:
        print(f"Error al procesar la URL {url}: {e}")
        return None

# Aplica la función de extracción a cada enlace en el DataFrame solo si la columna 'corpus' está vacía
df_final_investigadores_cargado2['corpus'] = df_final_investigadores_cargado2.apply(
    lambda row: obtener_corpus(row['Enlace'], row['corpus']) if not row['corpus'] else row['corpus'], axis=1)

# Ahora puedes revisar el DataFrame con la nueva columna 'corpus'
df_final_investigadores_cargado2

,Categoría,Autor,Título,Enlace,Imagen,Fecha,corpus
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,20 de Enero del 2025,"68%. Ese es el porcentaje que, según la encues..."
1,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...,10 de Enero del 2025,"Desde el punto de vista político, el nuevo año..."
2,Opinión,Claudio Alvarado R.,La agonía del octubrismo,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024,Las izquierdas jamás anticiparon cuán mal enve...
3,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024,Si la idea de una superior “escala de valores ...
4,Opinión,Claudio Alvarado R.,"""Habitar el cargo""",https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...,27 de Diciembre del 2024,"Señor Director: En su última cuenta pública, a..."
...,...,...,...,...,...,...,...
270,Claves e informes,Guillermo Pérez C.,¿Un Estado de bienestar para Chile?: 5 claves ...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,
271,Claves e informes,Guillermo Pérez C.,Mecanismos de democracia directa y nueva const...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,
272,Claves e informes,Guillermo Pérez C.,Descentralización y gobernadores regionales: 5...,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,
273,Claves e informes,Guillermo Pérez C.,Migración en Chile: 5 claves para el debate,https://multi-media.nyc3.digitaloceanspaces.co...,None,None,


In [16]:
df_final_investigadores_cargado2["Corpus"] = df_final_investigadores_cargado2["corpus"]

# Punto y coma

In [6]:
import pandas as pd

# Crear un diccionario con los datos de los artículos
data = {
    "Título": [
        "El legado del expresidente Piñera: balance preliminar",
        "Radiografía de las derechas",
        "Sol Serrano, historiadora: “Este país construyó nuevas élites y ahí el liceo tuvo un rol fundamental”.",
        "Punto y coma 11: \"Educación escolar: nuestra crisis\"",
        "El Estado en crisis: antecedentes y desafíos",
        "Pacto de clases y tregua de élites: el camino para salir de octubre",
        "El feminismo de la nueva izquierda bajo la lupa",
        "Yuval Levin: \"Pensar en la tradición de la que formas parte es la pregunta fundamental para todo conservador\"",
        "La (no) renovación: crisis de la derecha universitaria",
        "Punto y coma 10: Radiografía de las derechas",
        "Los infortunios de la democracia",
        "Revolución con banda sonora"
    ],
    "Categoría": [
        "Revista",
        "Revista",
        "Revista",
        "Revista",
        "Revista",
        "Revista",
        "Revista",
        "Revista",
        "Revista",
        "Revista",
        "ENSAYO",
        "ENSAYO"
    ],
    "Autor": [
        "Claudio Alvarado R.",
        "Joaquín Castillo V.",
        "Josefina Araos B.",
        "IES",
        "Daniel Mansuy H.",
        "Pablo Ortúzar M.",
        "Catalina Siles V.",
        "Guillermo Pérez C.",
        "María Asunción Poblete R.",
        "IES",
        "Chantal Delsol",
        "Marisol García"
    ],
    "Enlace": [
        "/revistapuntoycoma/el-legado-del-expresidente-piera-balance-preliminar",
        "/revistapuntoycoma/radiografa-de-las-derechas",
        "/revistapuntoycoma/sol-serrano-historiadora-este-pas-construy-nuevas-lites-y-ah-el-liceo-tuvo-un-rol-fundamental-",
        "/revistapuntoycoma/punto-y-coma-11-educacin-escolar-nuestra-crisis",
        "/revistapuntoycoma/el-estado-en-crisis-antecedentes-y-desafos",
        "/revistapuntoycoma/pacto-de-clases-y-tregua-de-lites-el-camino-para-salir-de-octubre-",
        "/revistapuntoycoma/el-feminismo-de-la-nueva-izquierda-bajo-la-lupa",
        "/revistapuntoycoma/yuval-levin-pensar-en-la-tradicin-de-la-que-formas-parte-es-la-pregunta-fundamental-para-todo-conservador",
        "/revistapuntoycoma/la-no-renovacin-crisis-de-la-derecha-universitaria",
        "/revistapuntoycoma/punto-y-coma-10-radiografia-de-las-derechas",
        "/revistapuntoycoma/los-infortunatos-de-la-democracia",
        "/revistapuntoycoma/revolucion-con-banda-sonora"
    ]
}

# Crear el DataFrame
df_pc = pd.DataFrame(data)

# Mostrar el DataFrame
df_pc

,Título,Categoría,Autor,Enlace
0,El legado del expresidente Piñera: balance pre...,Revista,Claudio Alvarado R.,/revistapuntoycoma/el-legado-del-expresidente-...
1,Radiografía de las derechas,Revista,Joaquín Castillo V.,/revistapuntoycoma/radiografa-de-las-derechas
2,"Sol Serrano, historiadora: “Este país construy...",Revista,Josefina Araos B.,/revistapuntoycoma/sol-serrano-historiadora-es...
3,"Punto y coma 11: ""Educación escolar: nuestra c...",Revista,IES,/revistapuntoycoma/punto-y-coma-11-educacin-es...
4,El Estado en crisis: antecedentes y desafíos,Revista,Daniel Mansuy H.,/revistapuntoycoma/el-estado-en-crisis-anteced...
5,Pacto de clases y tregua de élites: el camino ...,Revista,Pablo Ortúzar M.,/revistapuntoycoma/pacto-de-clases-y-tregua-de...
6,El feminismo de la nueva izquierda bajo la lupa,Revista,Catalina Siles V.,/revistapuntoycoma/el-feminismo-de-la-nueva-iz...
7,"Yuval Levin: ""Pensar en la tradición de la que...",Revista,Guillermo Pérez C.,/revistapuntoycoma/yuval-levin-pensar-en-la-tr...
8,La (no) renovación: crisis de la derecha unive...,Revista,María Asunción Poblete R.,/revistapuntoycoma/la-no-renovacin-crisis-de-l...
9,Punto y coma 10: Radiografía de las derechas,Revista,IES,/revistapuntoycoma/punto-y-coma-10-radiografia...


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def obtener_contenido(url):
    base_url = "https://www.ieschile.cl"
    full_url = base_url + url

    try:
        response = requests.get(full_url)
        response.raise_for_status()  # Lanza una excepción para códigos de error HTTP (4xx o 5xx)

        soup = BeautifulSoup(response.text, 'html.parser')

        # Selector para el contenedor principal del artículo
        contenedor_articulo = soup.find('div', class_='columna_info_body__IOoQA')  # Encuentra el contenedor principal
        if contenedor_articulo:  # Verifica si se encontró el contenedor
            parrafos = contenedor_articulo.find_all('p')  # Encuentra todos los <p> dentro del contenedor
            corpus = "\n".join(p.get_text() for p in parrafos)  # Une los textos de los párrafos con saltos de línea
        else:
            corpus = None

        # Extracción de la fecha (más robusta)
        fecha_element = soup.find('div', style=lambda x: x and 'font-family:InterLight' in x and 'font-size:11px' in x and 'margin-top:5px' in x)
        fecha = fecha_element.get_text().strip() if fecha_element else None

        return corpus, fecha

    except requests.exceptions.RequestException as e:
        print(f"Error al acceder a {full_url}: {e}")
        return None, None
    except AttributeError as e:  # Maneja errores si no encuentra los elementos
        print(f"Error al parsear {full_url}: {e}")
        return None, None
    except Exception as e:  # Captura cualquier otro error
        print(f"Error inesperado en {full_url}: {e}")
        return None, None


corpus_list = []
fecha_list = []

for index, row in df_pc.iterrows():
    enlace = row['Enlace']
    corpus, fecha = obtener_contenido(enlace)
    corpus_list.append(corpus)
    fecha_list.append(fecha)

df_pc['Corpus'] = corpus_list
df_pc['Fecha'] = fecha_list

df_pc

,Título,Categoría,Autor,Enlace,Corpus,Fecha
0,El legado del expresidente Piñera: balance pre...,Revista,Claudio Alvarado R.,/revistapuntoycoma/el-legado-del-expresidente-...,Pocas horas después de la trágica muerte de Se...,21 de Enero del 2025
1,Radiografía de las derechas,Revista,Joaquín Castillo V.,/revistapuntoycoma/radiografa-de-las-derechas,Artículo escrito por Joaquín Castillo y María ...,21 de Enero del 2025
2,"Sol Serrano, historiadora: “Este país construy...",Revista,Josefina Araos B.,/revistapuntoycoma/sol-serrano-historiadora-es...,Entrevista de Josefina Araos publicada en Punt...,14 de Enero del 2025
3,"Punto y coma 11: ""Educación escolar: nuestra c...",Revista,IES,/revistapuntoycoma/punto-y-coma-11-educacin-es...,Pocos temas han gozado de tanto protagonismo e...,08 de Octubre del 2024
4,El Estado en crisis: antecedentes y desafíos,Revista,Daniel Mansuy H.,/revistapuntoycoma/el-estado-en-crisis-anteced...,"A estas alturas, es un lugar común recordar qu...",11 de Diciembre del 2024
5,Pacto de clases y tregua de élites: el camino ...,Revista,Pablo Ortúzar M.,/revistapuntoycoma/pacto-de-clases-y-tregua-de...,Artículo escrito por Pablo Ortúzar y publicado...,28 de Noviembre del 2024
6,El feminismo de la nueva izquierda bajo la lupa,Revista,Catalina Siles V.,/revistapuntoycoma/el-feminismo-de-la-nueva-iz...,Artículo escrito por Catalina Siles y Gabriela...,21 de Noviembre del 2024
7,"Yuval Levin: ""Pensar en la tradición de la que...",Revista,Guillermo Pérez C.,/revistapuntoycoma/yuval-levin-pensar-en-la-tr...,Participante habitual de los debates estadouni...,08 de Noviembre del 2024
8,La (no) renovación: crisis de la derecha unive...,Revista,María Asunción Poblete R.,/revistapuntoycoma/la-no-renovacin-crisis-de-l...,Artículo publicado en la edición Nº 10 de la r...,24 de Octubre del 2024
9,Punto y coma 10: Radiografía de las derechas,Revista,IES,/revistapuntoycoma/punto-y-coma-10-radiografia...,Son varios los motivos que justifican un análi...,16 de Abril del 2024


# Completo

In [17]:
dfs_final = [df_final_investigadores_cargado2, df_pc]

ies_final = pd.concat(dfs_final, ignore_index=True)
ies_final

,Categoría,Autor,Título,Enlace,Imagen,Fecha,corpus,Corpus
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,20 de Enero del 2025,"68%. Ese es el porcentaje que, según la encues...","68%. Ese es el porcentaje que, según la encues..."
1,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...,10 de Enero del 2025,"Desde el punto de vista político, el nuevo año...","Desde el punto de vista político, el nuevo año..."
2,Opinión,Claudio Alvarado R.,La agonía del octubrismo,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024,Las izquierdas jamás anticiparon cuán mal enve...,Las izquierdas jamás anticiparon cuán mal enve...
3,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...,30 de Diciembre del 2024,Si la idea de una superior “escala de valores ...,Si la idea de una superior “escala de valores ...
4,Opinión,Claudio Alvarado R.,"""Habitar el cargo""",https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...,27 de Diciembre del 2024,"Señor Director: En su última cuenta pública, a...","Señor Director: En su última cuenta pública, a..."
...,...,...,...,...,...,...,...,...
282,Revista,Guillermo Pérez C.,"Yuval Levin: ""Pensar en la tradición de la que...",/revistapuntoycoma/yuval-levin-pensar-en-la-tr...,NaN,08 de Noviembre del 2024,NaN,Participante habitual de los debates estadouni...
283,Revista,María Asunción Poblete R.,La (no) renovación: crisis de la derecha unive...,/revistapuntoycoma/la-no-renovacin-crisis-de-l...,NaN,24 de Octubre del 2024,NaN,Artículo publicado en la edición Nº 10 de la r...
284,Revista,IES,Punto y coma 10: Radiografía de las derechas,/revistapuntoycoma/punto-y-coma-10-radiografia...,NaN,16 de Abril del 2024,NaN,Son varios los motivos que justifican un análi...
285,ENSAYO,Chantal Delsol,Los infortunios de la democracia,/revistapuntoycoma/los-infortunatos-de-la-demo...,NaN,07 de Octubre del 2019,NaN,Artículo inédito de la filósofa francesa Chant...


In [19]:
def convertir_fecha(fecha_str):
    if pd.isna(fecha_str):  # Verifica si el valor es NaN
        return None  # Retorna None si es NaN

    for formato in ["%d de %B del %Y", "%d %b. %Y", "%d-%m-%Y"]:
        try:
            fecha_dt = pd.to_datetime(fecha_str, format=formato)
            return fecha_dt.strftime('%Y-%m-%d')
        except ValueError:
            pass
    return None

# Aplica la función a la columna "Fecha" para crear una nueva columna "Fecha_publicacion"
ies_final['Fecha_publicacion'] = ies_final['Fecha'].apply(convertir_fecha)

# Elimina la columna original "Fecha" si ya no la necesitas
ies_final.drop('Fecha', axis=1, inplace=True)

# Imprime el DataFrame actualizado
ies_final

,Categoría,Autor,Título,Enlace,Imagen,corpus,Corpus,Fecha_publicacion
0,Opinión,Claudio Alvarado R.,"Pensiones, empleo y ciudadanía",https://www.ieschile.cl/columna/pensiones-empl...,https://www.ieschile.cl/_next/image?url=https%...,"68%. Ese es el porcentaje que, según la encues...","68%. Ese es el porcentaje que, según la encues...",2025-01-20
1,Opinión,Claudio Alvarado R.,Elecciones 2025: La oportunidad histórica de l...,https://www.ieschile.cl/columna/elecciones-202...,https://www.ieschile.cl/_next/image?url=https%...,"Desde el punto de vista político, el nuevo año...","Desde el punto de vista político, el nuevo año...",2025-01-10
2,Opinión,Claudio Alvarado R.,La agonía del octubrismo,https://www.ieschile.cl/columna/la-agona-del-o...,https://www.ieschile.cl/_next/image?url=https%...,Las izquierdas jamás anticiparon cuán mal enve...,Las izquierdas jamás anticiparon cuán mal enve...,2024-12-30
3,Opinión,Claudio Alvarado R.,Ni Allende ni Aylwin,https://www.ieschile.cl/columna/ni-allende-ni-...,https://www.ieschile.cl/_next/image?url=https%...,Si la idea de una superior “escala de valores ...,Si la idea de una superior “escala de valores ...,2024-12-30
4,Opinión,Claudio Alvarado R.,"""Habitar el cargo""",https://www.ieschile.cl/columna/habitar-el-cargo,https://www.ieschile.cl/_next/image?url=https%...,"Señor Director: En su última cuenta pública, a...","Señor Director: En su última cuenta pública, a...",2024-12-27
...,...,...,...,...,...,...,...,...
282,Revista,Guillermo Pérez C.,"Yuval Levin: ""Pensar en la tradición de la que...",/revistapuntoycoma/yuval-levin-pensar-en-la-tr...,NaN,NaN,Participante habitual de los debates estadouni...,2024-11-08
283,Revista,María Asunción Poblete R.,La (no) renovación: crisis de la derecha unive...,/revistapuntoycoma/la-no-renovacin-crisis-de-l...,NaN,NaN,Artículo publicado en la edición Nº 10 de la r...,2024-10-24
284,Revista,IES,Punto y coma 10: Radiografía de las derechas,/revistapuntoycoma/punto-y-coma-10-radiografia...,NaN,NaN,Son varios los motivos que justifican un análi...,2024-04-16
285,ENSAYO,Chantal Delsol,Los infortunios de la democracia,/revistapuntoycoma/los-infortunatos-de-la-demo...,NaN,NaN,Artículo inédito de la filósofa francesa Chant...,2019-10-07


In [20]:
ies_final.to_excel("IES.xlsx", index=False)